In [1]:
# !pip install transformers
# !wget https://github.com/t-davidson/hate-speech-and-offensive-language/raw/master/data/labeled_data.csv
# Put me under data/

In [2]:
# import btorch

In [3]:
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
# from tqdm.notebook import tqdm
import pandas as pd
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoModelForSequenceClassification,AutoModel
from transformers import AutoTokenizer, AutoConfig
from sklearn.utils import shuffle
import sklearn
import random

seed = 888
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

Using cuda


In [4]:
# path of data and the name of pretrained weights
# path = '../input/nlpdata/labeled_data.csv'
path = './data/labeled_data.csv'

In [5]:
df = pd.read_csv(path, index_col = 0)
df = shuffle(df)
df.describe()

,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [6]:
# 364 & 11176, Just to make sure the experiment is reproducible
df.iloc[0:2] 

,count,hate_speech,offensive_language,neither,class,tweet
364,3,1,2,0,1,"""@likemicah: Y'all dumb ass niggas be so press..."
11176,3,0,3,0,1,I snorted shit with more girls in my presence ...


In [7]:
df['class'].to_numpy()

array([1, 1, 1, ..., 1, 2, 2], dtype=int64)

In [8]:
# btorch.utils.get_class_weight(df['class'].to_numpy(), 'sklearn')

In [9]:
# split data into train and test
train_data = df.sample(frac = 0.8)
valid_data = df.drop(train_data.index)
test_data = valid_data.sample(frac = 0.5)
valid_data = valid_data.drop(test_data.index)

display(train_data.head())
print("===================================")
display(test_data.head())
print(train_data.shape)
print(test_data.shape)

,count,hate_speech,offensive_language,neither,class,tweet
5278,3,0,0,3,2,@WebAntOnYT btw Jim crow laws were supported b...
8208,3,0,3,0,1,Before I let a pussy cut my hair I just will l...
16962,3,0,0,3,2,RT @NASAWatch: .@tweetsoutloud we found your b...
20491,3,0,2,1,1,RT @sorryimalex: idkmadz tbh I've seen you in ...
4523,3,0,3,0,1,"@PaulConroy @habeshasuperman oh, it's coming u..."


,count,hate_speech,offensive_language,neither,class,tweet
4883,3,1,0,2,2,"@StarbucksSanae Are you sure about that, slant..."
2946,3,0,3,0,1,@Coley_Cee bitch dats yo sahara desert ass
11082,3,1,2,0,1,I never seen a Asian or Chinese bitch pregnant...
22484,3,0,3,0,1,"This bitch wanna smoke all of my weed,"
1305,3,0,0,3,2,&#8220;@GianneNichole: Lakers really trash.&#8...


(19826, 6)
(2478, 6)


# Build Dataset

The data are stored as a CSV and as a pickled pandas dataframe (Python 2.7). Each data df contains 5 columns:

count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

hate_speech = number of CF users who judged the tweet to be hate speech.

offensive_language = number of CF users who judged the tweet to be offensive.

neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, model_name=None,  train = True, device='cuda'):
        super(Dataset, self).__init__()
        self.df = df
        self.device = device
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        device = self.device
        count = torch.LongTensor([self.df.iloc[idx]['count']])
        hate_speech = torch.LongTensor([self.df.iloc[idx]['hate_speech']])
        offensive_language = torch.LongTensor([self.df.iloc[idx]['offensive_language']])
        neither = torch.LongTensor([self.df.iloc[idx]['neither']])
        target = torch.LongTensor([self.df.iloc[idx]['class']])
        tweet = self.df.iloc[idx]['tweet']
        return (count.to(device), hate_speech.to(device), offensive_language.to(device)
                , neither.to(device), target.to(device), tweet)

In [11]:
dataset = Dataset(df, device=device)
dataloader = torch.utils.data.DataLoader(dataset, batch_size= 6, shuffle=True)
output = next(iter(dataloader))
print(output)

[tensor([[3],
        [3],
        [3],
        [3],
        [3],
        [3]], device='cuda:0'), tensor([[0],
        [3],
        [0],
        [0],
        [0],
        [0]], device='cuda:0'), tensor([[3],
        [0],
        [3],
        [0],
        [3],
        [0]], device='cuda:0'), tensor([[0],
        [0],
        [0],
        [3],
        [0],
        [3]], device='cuda:0'), tensor([[1],
        [0],
        [1],
        [2],
        [1],
        [2]], device='cuda:0'), ('RT @BCniggas: I know plenty of hoes in love with they niggas but it be the same bitch fuckin niggas', 'fuck ray rice ,someone crack that bitch', 'Yo bitch she boof she got that throwbac pussy !', "RT @UFlorida: @GatorZoneFB It's a bird...It's a plane...It's Jake McGee! http://t.co/WVdLScoDvw", 'RT @CourtneyyKay: Cruising down the street in my 64, jockin the bitches, slappin the hoes', 'Early bird gets the worm! &#128027;&#128036;')]


# Utils

In [12]:
def number_params(model, exclude_freeze=False):
    """calculate the number of parameters in a model

    Args:
        model (nn.Module): PyTorch model
        exclude_freeze (bool, optional): Whether to count the frozen layer. Defaults to False.
    """
    pp = 0
    for p in list(model.parameters()):
        if exclude_freeze and p.requires_grad is False:
            continue
        nn = 1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

def finetune(
        model: nn.Module,
        base_lr: float,
        groups,
        ignore_the_rest: bool = False,
        raw_query: bool = False,
        regex=False):
    """ This is something call per-parameter options

    Separate out the finetune parameters with a learning rate for each layers of parameters
    This function only support setting a different learning rate for each layer's arameter.
    Depending on the optimizer, you can set extra parameter for that layer for the optmizer -> See Notes 
    If you freeze layer using this function and want to unfreeze it later:
    See https://discuss.pytorch.org/t/correct-way-to-freeze-layers/26714/2

    Args:
        model (nn.Module): Pytorch Model
        base_lr (float): learning rate of all layers
        groups (Dict[str, float]): key is `name` of layers, value is the `extra_lr` (or False).
          all layers that contains that `name` will have `lr` of base_lr*extra_lr.
          it uses fnmatch|regex to check whether a layer contains that `name`.
          fnmatch is matching structure like `layer1*`, `layer?.conv?.`, `*conv2*`, etc...
          regex is the comman regex matching.
          Hence, `name` here is either fnmatch or regex expression if using raw_query.
          If `float` is False: those layers with `name` will be freeze. 
          In particular, they will not be included in the return output and require_grad will be set to False
        ignore_the_rest (bool, optional): Include the remaining layer that are not stated in `grouprs` or not. Defaults to False.
        raw_query (bool, optional): Modify the keys of `groups` as f'*{key}*' if False. Only useful when `regex=False`
          Do not do any modification to the keys of `groups` if True. Defaults to False.
        regex (bool, optional): Use regex instead of fnmatch on keys of groups. Defaults to False.
          This will overrride raw_query to True. 
          Notice: `regex=False` is depracted

    Returns:
        List[Dict[str, Union[float, Iterable]]]: list of dict that has two or more key-value pair.
          The first one is feature generation layers. [those layers must start with `features` name] <usually is backbone>
            is a dict['params':list(model.parameters()), 'names':list(`layer's name`), 'query':query, 'lr':base_lr*groups[groups.keys()]]
          The remaining are all others layer. [all others params for last one, if ignore_the_rest = False]
            is a dict['params':list(model.parameters()), 'names':list(`layer's name`), 'lr':base_lr]

    Examples:
        >>> model = models.resnet50()
        >>> # all layers that has name start with `layer1 and layer2` will have learning rate `0.001*0.01`
        >>> # all layers that has name start with `layer3` will be froozen`
        >>> # all layers that has name start with `layer4` will have learning rate `0.001*0.001`
        >>> # for all other layers will have the base_lr `0.001`
        >>> model_params = finetune(model, base_lr=0.001, groups={'^layer[1-2].*': 0.01, '^layer3.*': False, '^layer4.*': 0.001}, regex=True)
        >>> # setting extra parameter (other than learning rate) for that optimizer
        >>> # the second param_group `layer4` will have weight_decay 1e-2
        >>> model_params[1]['weight_decay'] = 1e-2
        >>> # init optimizer with the above setting
        >>> # the argument under `torch.optim.SGD` will be overrided by finetune() if they exist.
        >>> # For example, all model_params will have weight_decay=5e-3 except model_params[1]
        >>> optimizer = torch.optim.SGD(model_params, momentum=0.9, lr=0.1, weight_decay=5e-3)
    """
    if regex:
        raw_query = True
    else:
        warnings.warn("regex=False is deprecated; use regex=True", DeprecationWarning)
    # Deal with Freeze Later
    freeze_group = dict()
    freeze = False
    for k,v in groups.items():
        if v is False:
            freeze_group[k] = 1
            freeze=True
    for k in freeze_group.keys():
        del groups[k]
    freeze_group = "(" + ")|(".join(freeze_group) + ")"

    parameters = [
        dict(params=[],
             names=[],
             query=query if raw_query else '*' + query + '*',
             lr = lr * base_lr,
             initial_lr = lr * base_lr) for query, lr in groups.items()
    ]
    rest_parameters = dict(params=[], names=[], lr=base_lr, initial_lr=base_lr)
    for k, v in model.named_parameters():
        rest = 0
        if freeze and regex and re.match(freeze_group, k):
            v.requires_grad = False
            continue
        for group in parameters:
            if not regex and fnmatch(k, group['query']):
                group['params'].append(v)
                group['names'].append(k)
                rest = 1
                break
            elif regex and re.compile(group['query']).search(k):
                group['params'].append(v)
                group['names'].append(k)
                rest = 1
                break
        if rest == 0:
            rest_parameters['params'].append(v)
            rest_parameters['names'].append(k)

    if not ignore_the_rest:
        parameters.append(rest_parameters)
    for group in parameters:
        group['params'] = iter(group['params'])
    return parameters

# Model

In [13]:
class LanguageModel(nn.Module):
    
    def __init__(self, model_name, num_labels = 3, freeze_pretrained=False):
        super(LanguageModel, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if freeze_pretrained:
            print("You are freezing the BERT")
            for name, p in self.model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
        print(f"Total number of params: {number_params(self.model)}")
        print(f"Total number of trainable params: {number_params(self.model, exclude_freeze=True)}")

    def forward(self, src, has_mask=False):
        # print(src)
        output = self.tokenizer(src, padding=True, truncation=True, max_length=50)
        output = torch.LongTensor(output['input_ids']).to(device)
        if has_mask == True:
            attention_mask=(output != 0).float() # here `0` is the <pad> token, i guess
            output = self.model(output, attention_mask=attention_mask)
        else:
            output = self.model(output)
        return output

In [14]:
class LastAttnModel(nn.Module):
    """
    Use the [CLS] as query and all other output as key and values.
    Pass it to a Multi-Head Attention, then a Linear classifier
    
    Args:
      auxiliary_head(list(int)): Only used when training
        - list of idx of hidden_layers that will be used as auxiliary_head. Here `idx` start from 1
        - See BertConfig['num_hidden_layers'] for total number of layers
        - EG: `auxiliary_head=[10,11,12]`.
      last_hidden_layer(int): Treat the output of this layer as last_hidden_layer

    Examples:
      tweet = iter(dataloader_train).next()[-1]
      modelA = LastAttnModel(pretrain_model, tokenizer).to(device)
      out, attn_weight = modelA(list(tweet))
      print(out.shape, attn_weight.shape) #torch.Size([32, 3]) torch.Size([32, 1, 49])
      
    Returns:
      list(tensor): the first tensor is the prediction, the second is the attention weight
    """
    
    def __init__(self, pretrain_model, tokenizer, 
                 last_attn_num_head = 8,
                 classifier_hidden_dim = 512, 
                 classifier_dropout = 0,
                 num_labels = 3, 
                 freeze_pretrained=False,
                 auxiliary_head=None,
                 last_hidden_layer=-1):
        super(LastAttnModel, self).__init__()
        self.pretrain_model = pretrain_model
        self.tokenizer = tokenizer
        self.auxiliary_head = auxiliary_head
        self.num_layers = len(pretrain_model.encoder.layer)
        self.last_hidden_layer = last_hidden_layer
        
        if freeze_pretrained:
            if self.auxiliary_head is not None:
              warnings.warn("freeze_pretrained and auxiliary_head set to True together is useless for training. Consider use `finetune()`")
            print("You are freezing the BERT pertrain")
            for name, p in self.pretrain_model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
        
        embed_size = pretrain_model.embeddings.word_embeddings.embedding_dim
        self.last_attn = nn.MultiheadAttention(embed_size, last_attn_num_head, batch_first=True)
        self.final_classifier = nn.Sequential(
            nn.Linear(embed_size, classifier_hidden_dim),
            nn.ReLU(),
            nn.Dropout(classifier_dropout),
            nn.Linear(classifier_hidden_dim, num_labels)
        )

        # deal with aux 
        if self.auxiliary_head is not None:
          self.aux_classifiers = nn.ModuleList()
          for i in self.auxiliary_head:
            self.aux_classifiers.append(nn.Sequential(
              nn.Linear(embed_size, classifier_hidden_dim),
              nn.ReLU(),
              nn.Dropout(classifier_dropout),
              nn.Linear(classifier_hidden_dim, num_labels)
            ))


        print(f"Total number of params: {number_params(self)}")
        print(f"Total number of trainable params: {number_params(self, exclude_freeze=True)}")
    def forward(self, src, has_mask=False):
        # print(src)
        out = []
        tokens = self.tokenizer(src, padding=True, truncation=True, max_length=50)
        inputs = torch.LongTensor(tokens['input_ids']).to(device)
        if has_mask == True:
            # attention_mask=(inputs != 0).float() # here `0` is the <pad> token, i guess
            attention_mask = torch.LongTensor(tokens['attention_mask']).to(device)
            pre_train_output = self.pretrain_model(inputs, attention_mask=attention_mask)
        else:
            pre_train_output = self.pretrain_model(inputs)
        # last_hidden = pre_train_output["last_hidden_state"]      # (N,T,E)
        last_hidden = pre_train_output["hidden_states"][self.last_hidden_layer]
        last_hidden_state_cls = last_hidden[:, 0, :].unsqueeze(1)   # (N,1,E)
        last_hidden_state_rest = last_hidden[:, 1:, :] # (N,T-1,E)
        atten_mask_pad = (inputs == 0)[:,1:] #(N,T-1)
        last_attn_out, last_attn_w = self.last_attn(last_hidden_state_cls, last_hidden_state_rest, last_hidden_state_rest,
                                                    key_padding_mask=atten_mask_pad) #(N,1,E), (N,1,T-1)
        last_attn_out = last_attn_out.squeeze(1) #(N,E)
        output = self.final_classifier(last_attn_out)
        out += [output, last_attn_w]
        ## auxiliary_head
        if self.auxiliary_head is not None:
          if "hidden_states" not in pre_train_output:
            raise Exception("Put `pre_train_output=True` in AutoConfig")
          for idx in range(len(self.aux_classifiers)):
            hidden_cls = pre_train_output["hidden_states"][self.auxiliary_head[idx-1]][:, 0, :] # (N,E)
            out.append(self.aux_classifiers[idx](hidden_cls))
        return out
    
# ## Usage
# tweet = iter(dataloader_train).next()[-1]
# modelA = LastAttnModel(pretrain_model, tokenizer).to(device)
# out, attn_weight = modelA(list(tweet))
# print(out.shape, attn_weight.shape)

# Initialization Stage 1

In [15]:
traindataset1 = Dataset(train_data)
dataloader_train1 = torch.utils.data.DataLoader(traindataset1, batch_size=32, shuffle=True)

validdataset1 = Dataset(valid_data)
dataloader_valid1 = torch.utils.data.DataLoader(validdataset1, batch_size=32, shuffle=False)

test_dataset1 = Dataset(test_data)
dataloader_test1 = torch.utils.data.DataLoader(test_dataset1, batch_size=32, shuffle=False)

lossfn1 = nn.CrossEntropyLoss().to(device)
# lossfn = nn.CrossEntropyLoss(weight=torch.tensor([5.7769, 0.4305, 1.9844])).to(device)
# model = LanguageModel(model_name, freeze_pretrained=False).to(device)
model_name1 = 'bert-base-uncased'
config1 = AutoConfig.from_pretrained(
    model_name1, 
    num_labels = 2,
    output_hidden_states = True,
    output_attention = False,
    hidden_dropout_prob = 0.2,
) 
print(config1)
pretrain_model1 = AutoModel.from_pretrained(
    model_name1,
    config = config1
).to(device)

tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
model1 = LastAttnModel(pretrain_model1, tokenizer1, freeze_pretrained=True, num_labels = 2,
                      classifier_dropout=0.2,
                      auxiliary_head=None,
                      last_hidden_layer=5).to(device)
# model_params = finetune(model, base_lr=1e-4, 
#                         groups={'^pretrain_model.encoder.layer.([0-7])\..*': False, 
#                                 '^pretrain_model.encoder.layer.([8-9]|1[012]).*': 0.01, 
#                                 '^pretrain_model.pooler.*': False,
#                                 '^pretrain_model.embeddings.*':False},
#                         regex=True)
optimizer1 = torch.optim.Adam(model1.parameters(), lr=1e-4, weight_decay=1e-3)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


You are freezing the BERT pertrain
Total number of params: 112239362
Total number of trainable params: 2757122


## DEBUG

In [16]:
# model_name = 'bert-base-uncased'
# config = AutoConfig.from_pretrained(
#     model_name, 
#     output_hidden_states = True,
#     output_attention = False
# ) 
# print(config)
# model = AutoModel.from_pretrained(
#     model_name,
#     config = config
# ).to(device)

# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
# tweet = iter(dataloader_train).next()[-1]
# tokens = tokenizer(list(tweet), padding=True, truncation=True, max_length=50)
# print(tokens.keys())
# inputs = torch.LongTensor(tokens['input_ids']).to(device)
# atten_mask = torch.LongTensor(tokens['attention_mask']).to(device)
# print(inputs.shape)
# hiddens = model(inputs)
# print(hiddens.keys())
# print(hiddens['last_hidden_state'].shape)

In [18]:
# (inputs[0]!=0).float()

In [19]:
# model.embeddings.word_embeddings.embedding_dim

In [20]:
# atten_mask_pad = (inputs == 0)
# print(atten_mask_pad.shape)
# print(atten_mask_pad[:,1:].shape)

# Train Stage 1

In [21]:
def train1(dataloader_train, dataloader_valid = None, dataloader_test = None, model1 = None, 
          optimizer = None, lossfn = None,  epochs = 10, has_mask = True):
    
    trainloss = []
    validloss = []
    testloss = []
    trainscore = []
    validscore = []
    testscore = []
    
    traincm = []
    validcm = []
    testcm = []

    for i in range(epochs):
        model1.train()
        averageloss = 0
        averagescore = 0
        for datas in tqdm(dataloader_train):
            count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
            
            # 0 = neither hate speech nor offensive language
            # 1 = either hate speech or offensive language
            target_new = torch.LongTensor([[0] if t == 2 else [1] for t in target]).to(device)
            
            optimizer.zero_grad()
            pred = model1(list(tweet), has_mask)
            loss = lossfn(pred[0], target_new.squeeze(1))
            # aux head
            for j in range(2, len(pred)):
              loss+=0.3*lossfn(pred[j], target_new.squeeze(1))
            loss.backward()
            optimizer.step()
            f1score = sklearn.metrics.f1_score(target_new.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
            averageloss += loss.item()/len(dataloader_train)
            averagescore += f1score/len(dataloader_train)
        trainloss.append(averageloss)
        trainscore.append(averagescore)
        if dataloader_valid is not None:
            model1.eval()
            averageloss = 0
            averagescore = 0
            for datas in tqdm(dataloader_valid):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                
                # 0 = neither hate speech nor offensive language
                # 1 = either hate speech or offensive language
                target_new = torch.LongTensor([[0] if t == 2 else [1] for t in target]).to(device)
                
                pred = model1(list(tweet), has_mask)
                loss = lossfn(pred[0], target_new.squeeze(1))
                f1score = sklearn.metrics.f1_score(target_new.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                averageloss += loss.item()/len(dataloader_valid)
                averagescore += f1score/len(dataloader_valid)
            validloss.append(averageloss)
            validscore.append(averagescore)
        if dataloader_test is not None:
            model1.eval()
            averageloss = 0
            averagescore = 0
            for datas in tqdm(dataloader_test):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                
                # 0 = neither hate speech nor offensive language
                # 1 = either hate speech or offensive language
                target_new = torch.LongTensor([[0] if t == 2 else [1] for t in target]).to(device)
                
                pred = model1(list(tweet), has_mask)
                loss = lossfn(pred[0], target_new.squeeze(1))
                f1score = sklearn.metrics.f1_score(target_new.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                averageloss += loss.item()/len(dataloader_test)
                averagescore += f1score/len(dataloader_test)
            testloss.append(averageloss)
            testscore.append(averagescore)
            
            
        print(f"epoch: {i+1}")
        print(f"train loss: {trainloss[-1]}, train f1score: {trainscore[-1]}")
        if dataloader_valid is not None:
            print(f"validation loss: {validloss[-1]}, validation f1score: {validscore[-1]}")
        if dataloader_test is not None:
            print(f"test loss: {testloss[-1]}, test f1score: {testscore[-1]}")

    return trainloss, validloss, testloss, trainscore, validscore, testscore

In [22]:
epochs1 = 20
trainloss1, validloss1, testloss1, trainscore1, validscore1, testscore1 = train1(
    dataloader_train1, dataloader_valid1, dataloader_test1,
    model1 = model1,
    optimizer = optimizer1, lossfn = lossfn1, epochs = epochs1, has_mask = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:09<00:00,  8.28it/s]


epoch: 1
train loss: 0.14979061067044266, train f1score: 0.9336889490668375
validation loss: 0.10726041886477898, validation f1score: 0.957026564288009
test loss: 0.10960878220458445, test f1score: 0.9573215041028394


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:09<00:00,  8.10it/s]


epoch: 2
train loss: 0.10018462354257215, train f1score: 0.9611723125932705
validation loss: 0.1083862581409705, validation f1score: 0.9555316146770725
test loss: 0.10841767424836947, test f1score: 0.9573881531685822


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:09<00:00,  8.32it/s]


...

epoch: 18
train loss: 0.03432367644297387, train f1score: 0.985065913774087
validation loss: 0.17536417549686467, validation f1score: 0.9520648255844363
test loss: 0.1686706265695064, test f1score: 0.9531854987072717


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:10<00:00,  7.76it/s]


epoch: 19
train loss: 0.03518129933267292, train f1score: 0.9846050386289664
validation loss: 0.14812211007273823, validation f1score: 0.957023702062327
test loss: 0.15462304345251487, test f1score: 0.9554984432326289


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:09<00:00,  8.26it/s]

epoch: 20
train loss: 0.031038437609171403, train f1score: 0.9861789266019946
validation loss: 0.15884340264258354, validation f1score: 0.9508958745970569
test loss: 0.15633568211566082, test f1score: 0.9549776779973497


In [23]:
trainloss1

[0.14979061067044266,
 0.10018462354257215,
 0.09202000658316985,
 0.08283704971451494,
 0.07628191690676062,
 0.07051364952818519,
 0.06307000820956854,
 0.058878085353682136,
 0.0555835037802031,
 0.05248453440107103,
 0.04738443526063834,
 0.04443426501351386,
 0.04228087439846754,
 0.041041821803880725,
 0.03736743032131926,
 0.03560971393200171,
 0.03882596704446062,
 0.03432367644297387,
 0.03518129933267292,
 0.031038437609171403]

In [24]:
validloss1

[0.10726041886477898,
 0.1083862581409705,
 0.10432043170126584,
 0.11366790375457361,
 0.12630009364623287,
 0.13864749460928863,
 0.12323950453756902,
 0.12243100257369521,
 0.11609552684837043,
 0.13224543657344887,
 0.13620633535454263,
 0.13015654073574415,
 0.15135527977373647,
 0.14380276210319543,
 0.17006568043516615,
 0.15671424704562056,
 0.15558650512367686,
 0.17536417549686467,
 0.14812211007273823,
 0.15884340264258354]

In [25]:
testloss1

[0.10960878220458445,
 0.10841767424836947,
 0.10520326635704783,
 0.11912514939784814,
 0.12757198941649103,
 0.1346715210716073,
 0.12359811118147215,
 0.12434970503314759,
 0.12420538414078648,
 0.12795496846322352,
 0.1353236912191511,
 0.12349964137702515,
 0.15188027260592207,
 0.1426421739399815,
 0.17096445903156662,
 0.1464689866824745,
 0.14273544768534174,
 0.1686706265695064,
 0.15462304345251487,
 0.15633568211566082]

In [26]:
trainscore1

[0.9336889490668375,
 0.9611723125932705,
 0.9655884545280914,
 0.969954804119201,
 0.9723029400280312,
 0.9732122773575447,
 0.975583652066253,
 0.9771028175012676,
 0.978594263091581,
 0.979581327858186,
 0.9806456914912757,
 0.9838535020248055,
 0.9834539501155998,
 0.9829826685948024,
 0.984272990536984,
 0.9847898937970024,
 0.9841594687399614,
 0.985065913774087,
 0.9846050386289664,
 0.9861789266019946]

In [27]:
validscore1

[0.957026564288009,
 0.9555316146770725,
 0.9565913849133489,
 0.9589319907229515,
 0.9582615769498036,
 0.9509963425005151,
 0.9587199840781723,
 0.9524811100611483,
 0.9487079246544438,
 0.9556436968579249,
 0.9458791894607448,
 0.9482584540670904,
 0.9539405152503652,
 0.9545875243896446,
 0.9468090656167948,
 0.9517230579709405,
 0.9500426217837971,
 0.9520648255844363,
 0.957023702062327,
 0.9508958745970569]

In [28]:
testscore1

[0.9573215041028394,
 0.9573881531685822,
 0.9598588159235296,
 0.960048669271405,
 0.9559714655692211,
 0.9568538243543364,
 0.9579547986490029,
 0.9555048647110438,
 0.9468179213782166,
 0.9571286016884881,
 0.9483087303230727,
 0.9527276841012294,
 0.9552055294910252,
 0.9564896701502049,
 0.9517850452886367,
 0.9548502042948914,
 0.9547701741213658,
 0.9531854987072717,
 0.9554984432326289,
 0.9549776779973497]

# In case You want know what is `pred`  
# Check the attension weight to swear word

In [29]:
for datas in dataloader_valid1:
    model1.eval()
    count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
    pred, w = model1(list(tweet), True)
    break

In [30]:
pred

tensor([[ -3.6735,   3.6505],
        [-10.4447,  10.4085],
        [ -5.5244,   5.4978],
        [  1.2984,  -1.3067],
        [  2.9445,  -2.9613],
        [  7.1286,  -7.1529],
        [ -3.9050,   3.8821],
        [ -4.5350,   4.5106],
        [  0.8427,  -0.8518],
        [ -3.7783,   3.7554],
        [ -5.4188,   5.3933],
        [ -5.3347,   5.3089],
        [ -3.0732,   3.0517],
        [ -1.5118,   1.4934],
        [ -6.8288,   6.7996],
        [ -7.7434,   7.7133],
        [ -6.4596,   6.4312],
        [ -4.7019,   4.6772],
        [ -8.7932,   8.7598],
        [ -8.0605,   8.0288],
        [ -3.3683,   3.3459],
        [  3.1295,  -3.1500],
        [ -4.3012,   4.2774],
        [ -7.9652,   7.9343],
        [ -7.0695,   7.0403],
        [ -8.1264,   8.0946],
        [  0.8340,  -0.8383],
        [ -8.0964,   8.0651],
        [ -7.2449,   7.2146],
        [ -3.0113,   2.9892],
        [ -5.8623,   5.8355],
        [ -6.0737,   6.0459]], device='cuda:0', grad_fn=<AddmmBackward

In [31]:
idx = 6
print(f"Prediction: {pred[idx].argmax().item()}. Ground-Truth: {target[idx].item()}")
print(tweet[idx])
w_pure = w[idx][w[idx] !=0 ][1:].cpu().detach().numpy()
tokens = tokenizer1.tokenize(tweet[idx])
full = list(zip(tokens, w_pure))
display(full)
# here `w_pure.sum() != 1` because the <cls> token score is not included, so will be small then one.

Prediction: 1. Ground-Truth: 1
RT @YoungCoke__: "@1stBlocJeremiah: @1stBlock_Rody yu ah hoe"lmfao,


[('rt', 0.0074906293),
 ('@', 0.012355257),
 ('young', 0.008500561),
 ('##co', 0.049675375),
 ('##ke', 0.008071665),
 ('_', 0.007797317),
 ('_', 0.020686295),
 (':', 0.01042659),
 ('"', 0.0077177677),
 ('@', 0.024243329),
 ('1st', 0.028999342),
 ('##bl', 0.0089940075),
 ('##oc', 0.029942203),
 ('##jer', 0.032848917),
 ('##emia', 0.017617648),
 ('##h', 0.04548478),
 (':', 0.00777625),
 ('@', 0.03684309),
 ('1st', 0.02670883),
 ('##block', 0.008195631),
 ('_', 0.058038127),
 ('rod', 0.0454424),
 ('##y', 0.052891042),
 ('yu', 0.08151081),
 ('ah', 0.2260769),
 ('ho', 0.017454928),
 ('##e', 0.008713197),
 ('"', 0.0316812),
 ('l', 0.0074782753),
 ('##m', 0.019092595),
 ('##fa', 0.014751794),
 ('##o', 0.00981262),
 (',', 0.01100533)]

In [32]:
# combine tokens to word by handling '##'
out = []
out.append(list(full[0]))
for i in range(1, len(full)):
    if full[i][0][:2] == '##':
        out[-1][0] += full[i][0][2:]
        out[-1][1] += full[i][1]
    else:
        out.append(list(full[i]))
out

[['rt', 0.0074906293],
 ['@', 0.012355257],
 ['youngcoke', 0.0662476],
 ['_', 0.007797317],
 ['_', 0.020686295],
 [':', 0.01042659],
 ['"', 0.0077177677],
 ['@', 0.024243329],
 ['1stblocjeremiah', 0.1638869],
 [':', 0.00777625],
 ['@', 0.03684309],
 ['1stblock', 0.03490446],
 ['_', 0.058038127],
 ['rody', 0.09833344],
 ['yu', 0.08151081],
 ['ah', 0.2260769],
 ['hoe', 0.026168125],
 ['"', 0.0316812],
 ['lmfao', 0.051135283],
 [',', 0.01100533]]

In [33]:
# score by attention score
sorted(out, key=lambda out: out[1], reverse=True)

[['ah', 0.2260769],
 ['1stblocjeremiah', 0.1638869],
 ['rody', 0.09833344],
 ['yu', 0.08151081],
 ['youngcoke', 0.0662476],
 ['_', 0.058038127],
 ['lmfao', 0.051135283],
 ['@', 0.03684309],
 ['1stblock', 0.03490446],
 ['"', 0.0316812],
 ['hoe', 0.026168125],
 ['@', 0.024243329],
 ['_', 0.020686295],
 ['@', 0.012355257],
 [',', 0.01100533],
 [':', 0.01042659],
 ['_', 0.007797317],
 [':', 0.00777625],
 ['"', 0.0077177677],
 ['rt', 0.0074906293]]

# Initialization Stage 2

In [34]:
train_data2 = train_data.copy(deep=True)
train_data2 = train_data2.drop(train_data2.index[train_data2['class'] == 2].tolist())
train_data2 = train_data2.reset_index(drop=True)

valid_data2 = valid_data.copy(deep=True)
valid_data2 = valid_data2.drop(valid_data2.index[valid_data2['class'] == 2].tolist())
valid_data2 = valid_data2.reset_index(drop=True)

test_data2 = test_data.copy(deep=True)
test_data2 = test_data2.drop(test_data2.index[test_data2['class'] == 2].tolist())
test_data2 = test_data2.reset_index(drop=True)

traindataset2 = Dataset(train_data2)
validdataset2 = Dataset(valid_data2)
testdataset2 = Dataset(test_data2)

dataloader_train2 = torch.utils.data.DataLoader(traindataset2, batch_size=32, shuffle=True)
dataloader_valid2 = torch.utils.data.DataLoader(validdataset2, batch_size=32, shuffle=False)
dataloader_test2 = torch.utils.data.DataLoader(testdataset2, batch_size=32, shuffle=False)

test_data3 = test_data.copy(deep=True)
testdataset3 = Dataset(test_data3)
dataloader_test3 = torch.utils.data.DataLoader(testdataset3, batch_size=32, shuffle=False)

lossfn2 = nn.CrossEntropyLoss().to(device)
# lossfn = nn.CrossEntropyLoss(weight=torch.tensor([5.7769, 0.4305, 1.9844])).to(device)
# model = LanguageModel(model_name, freeze_pretrained=False).to(device)
model_name2 = 'bert-base-uncased'
config2 = AutoConfig.from_pretrained(
    model_name2, 
    num_labels = 2,
    output_hidden_states = True,
    output_attention = False,
    hidden_dropout_prob = 0.2,
) 
print(config2)
pretrain_model2 = AutoModel.from_pretrained(
    model_name2,
    config = config2
).to(device)

tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2 = LastAttnModel(pretrain_model2, tokenizer2, freeze_pretrained=True, num_labels = 2,
                      classifier_dropout=0.2,
                      auxiliary_head=None,
                      last_hidden_layer=5).to(device)
# model_params = finetune(model, base_lr=1e-4, 
#                         groups={'^pretrain_model.encoder.layer.([0-7])\..*': False, 
#                                 '^pretrain_model.encoder.layer.([8-9]|1[012]).*': 0.01, 
#                                 '^pretrain_model.pooler.*': False,
#                                 '^pretrain_model.embeddings.*':False},
#                         regex=True)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-4, weight_decay=1e-3)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


You are freezing the BERT pertrain
Total number of params: 112239362
Total number of trainable params: 2757122


# Train Stage 2

In [35]:
def train2(dataloader_train2, dataloader_valid2 = None, dataloader_test2 = None, dataloader_test3 = None, model1 = None, model2 = None,
          optimizer = None, lossfn = None,  epochs = 10, has_mask = True):
    
    trainloss = []
    validloss = []
    testloss = []
    trainscore = []
    validscore = []
    testscore = []
    
    testcounter_all = []
    testcounter_either = []
    testscores = []
    
    for i in range(epochs):
        model2.train()
        averageloss = 0
        averagescore = 0
        for datas in tqdm(dataloader_train2):
            count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
            
            optimizer.zero_grad()
            pred = model2(list(tweet), has_mask)
            loss = lossfn(pred[0], target.squeeze(1))
            # aux head
            for j in range(2, len(pred)):
              loss+=0.3*lossfn(pred[j], target.squeeze(1))
            loss.backward()
            optimizer.step()
            f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
            averageloss += loss.item()/len(dataloader_train2)
            averagescore += f1score/len(dataloader_train2)
        trainloss.append(averageloss)
        trainscore.append(averagescore)
        
        if dataloader_valid2 is not None:
            model2.eval()
            averageloss = 0
            averagescore = 0
            for datas in tqdm(dataloader_valid2):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                
                pred = model2(list(tweet), has_mask)
                loss = lossfn(pred[0], target.squeeze(1))
                f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                averageloss += loss.item()/len(dataloader_valid2)
                averagescore += f1score/len(dataloader_valid2)
            validloss.append(averageloss)
            validscore.append(averagescore)
            
        if dataloader_test2 is not None:
            model2.eval()
            averagescore = 0
            counter = 0
            for datas in tqdm(dataloader_test2):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                
                pred = model2(list(tweet), has_mask)
                loss = lossfn(pred[0], target.squeeze(1))
                f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                averageloss += loss.item()/len(dataloader_test2)
                averagescore += f1score/len(dataloader_test2)
            testloss.append(averageloss)
            testscore.append(averagescore)
            
        if dataloader_test3 is not None:
            model2.eval()
            averagescore = 0
            counter_all = 0
            counter_either = 0
            for datas in tqdm(dataloader_test3):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                
                model1.eval()
                pred1 = model1(list(tweet), True) # predict with stage 1 model
                pred1 = pred1[0].argmax(-1)
                pred1 = list(map(bool, pred1))
                
                target_new = target[pred1].to(device)
                
                tweet_new = [t if pred1[i] else None for i, t in enumerate(tweet)]
                tweet_new = list(filter(None, tweet_new))
                tweet_new = tuple(tweet_new)
                
                counter_all += len(tweet)
                
                pred2 = model2(list(tweet_new), has_mask)
                
                for idx3 in range(len(pred2[0])):
                    
                    f1score = sklearn.metrics.f1_score(target_new[idx3].cpu().numpy(), pred2[0].argmax(-1)[idx3].unsqueeze(0).cpu().numpy(), average = 'weighted')
                    counter_either += 1
                    averagescore += f1score
                    
            testcounter_all.append(counter_all)
            testcounter_either.append(counter_either)
            testscores.append(averagescore/testcounter_either[-1])
        
        
        print(f"epoch: {i+1}")
        print(f"train loss: {trainloss[-1]}, train f1score: {trainscore[-1]}")
        if dataloader_valid2 is not None:
            print(f"validation loss: {validloss[-1]}, validation f1score: {validscore[-1]}")
        if dataloader_test2 is not None:
            print(f"test loss: {testloss[-1]}, test f1score: {testscore[-1]}")
        if dataloader_test3 is not None:
            print(f"final test f1score: {testscores[-1]}, {testcounter_either[-1]}/{testcounter_all[-1]}")
        
            
    return trainloss, validloss, testloss, trainscore, validscore, testscore, testscores

In [36]:
epochs2 = 20
trainloss2, validloss2, testloss2, trainscore2, validscore2, testscore2, testscores = train2(
    dataloader_train2, dataloader_valid2, dataloader_test2, dataloader_test3,
    model1 = model1, model2 = model2,
    optimizer = optimizer2, lossfn = lossfn2, epochs = epochs2, has_mask = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:16<00:00,  4.61it/s]


epoch: 1
train loss: 0.19345033957890065, train f1score: 0.911747642585105
validation loss: 0.17875599872786552, validation f1score: 0.9258048982993812
test loss: 0.36609898795811197, test f1score: 0.9182416451269046
final test f1score: 0.8928909952606635, 2110/2478


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:17<00:00,  4.55it/s]


epoch: 2
train loss: 0.16588398793445125, train f1score: 0.9232315490981621
validation loss: 0.17153550492366776, validation f1score: 0.924988857184504
test loss: 0.3521895806400359, test f1score: 0.9171291045829011
final test f1score: 0.8962085308056872, 2110/2478


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:17<00:00,  4.52it/s]


...

epoch: 18
train loss: 0.07409733127623926, train f1score: 0.9669843402341298
validation loss: 0.2527930217256653, validation f1score: 0.9169673065074467
test loss: 0.5257001550209789, test f1score: 0.9108533303583476
final test f1score: 0.8876777251184834, 2110/2478


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:16<00:00,  4.81it/s]


epoch: 19
train loss: 0.07362104833483431, train f1score: 0.9681194320048382
validation loss: 0.28585269609175157, validation f1score: 0.9212713900229901
test loss: 0.60671774088824, test f1score: 0.9071792230871841
final test f1score: 0.8829383886255924, 2110/2478


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:16<00:00,  4.79it/s]

epoch: 20
train loss: 0.07113749089347589, train f1score: 0.9676954237321835
validation loss: 0.2524229860646301, validation f1score: 0.916556035838939
test loss: 0.5403365415186273, test f1score: 0.9063163437772063
final test f1score: 0.8881516587677725, 2110/2478


In [37]:
trainloss2

[0.19345033957890065,
 0.16588398793445125,
 0.15866907913772055,
 0.15096000777178467,
 0.14363724109536205,
 0.13350474474547325,
 0.1261653406590641,
 0.11763438142207987,
 0.11113020356712046,
 0.10242915013398926,
 0.09988029898900565,
 0.0935307705360681,
 0.09143234416278431,
 0.08763914010460415,
 0.08041670344374685,
 0.08025556306269876,
 0.07966211287380562,
 0.07409733127623926,
 0.07362104833483431,
 0.07113749089347589]

In [38]:
validloss2

[0.17875599872786552,
 0.17153550492366776,
 0.1761324374238029,
 0.1888341586382012,
 0.1858036730773165,
 0.17708812539058272,
 0.17982847438543104,
 0.19222994867595844,
 0.2057357031153515,
 0.2133626615977846,
 0.22403574269264936,
 0.21046449727145955,
 0.2259934559406247,
 0.25221860217061476,
 0.2519320841311128,
 0.26245560075403773,
 0.26826016208906367,
 0.2527930217256653,
 0.28585269609175157,
 0.2524229860646301]

In [39]:
testloss2

[0.36609898795811197,
 0.3521895806400359,
 0.356980803045162,
 0.4082402364493289,
 0.38470922337353713,
 0.3720006041134406,
 0.37230786911448266,
 0.4025802674736813,
 0.43645750366638175,
 0.4461527899221631,
 0.4708566720664472,
 0.43141483580235,
 0.45952831694739865,
 0.5288807820304073,
 0.5261261239411946,
 0.5480923053552089,
 0.5547624434022899,
 0.5257001550209789,
 0.60671774088824,
 0.5403365415186273]

In [40]:
trainscore2

[0.911747642585105,
 0.9232315490981621,
 0.927821598184319,
 0.930691606092174,
 0.9370222802090351,
 0.9428420624492115,
 0.9449729659537174,
 0.9485392020185618,
 0.9531204697281761,
 0.9546318922385606,
 0.9537856793176093,
 0.9587591053914164,
 0.9602276097362495,
 0.9593330750559601,
 0.9642280743332838,
 0.9637460174766899,
 0.9656252391530057,
 0.9669843402341298,
 0.9681194320048382,
 0.9676954237321835]

In [41]:
validscore2

[0.9258048982993812,
 0.924988857184504,
 0.9131137817501886,
 0.9257431992586246,
 0.9217056017573693,
 0.9173224769362599,
 0.9230527940194653,
 0.9219023347783489,
 0.9203744766243311,
 0.9185998847801519,
 0.9176228589501041,
 0.9171036753085134,
 0.921322223612996,
 0.9185121321571742,
 0.9132210977918974,
 0.9166561144780981,
 0.9151437115575806,
 0.9169673065074467,
 0.9212713900229901,
 0.916556035838939]

In [42]:
testscore2

[0.9182416451269046,
 0.9171291045829011,
 0.9112565541410812,
 0.914857804138425,
 0.9151535032233576,
 0.911520786184359,
 0.9188543946779608,
 0.9149206124896895,
 0.9183045688430638,
 0.9185395873589997,
 0.912762937799887,
 0.9121236644476787,
 0.9178943855100608,
 0.9121429838540471,
 0.9154827063463996,
 0.9133101900379214,
 0.9123212679762145,
 0.9108533303583476,
 0.9071792230871841,
 0.9063163437772063]

In [43]:
testscores

[0.8928909952606635,
 0.8962085308056872,
 0.8985781990521327,
 0.8971563981042654,
 0.9009478672985782,
 0.8943127962085308,
 0.8995260663507109,
 0.8919431279620853,
 0.8966824644549763,
 0.8981042654028436,
 0.8914691943127963,
 0.895260663507109,
 0.895260663507109,
 0.8867298578199052,
 0.8957345971563981,
 0.8933649289099526,
 0.8933649289099526,
 0.8876777251184834,
 0.8829383886255924,
 0.8881516587677725]